In [11]:
import math
import piplite
await piplite.install('bqplot')
await piplite.install('ipywidgets')
import bqplot
from bqplot import pyplot as plt
import ipywidgets as widgets
import numpy as np
from scipy.integrate import solve_ivp

In [12]:
#Variables

n = 2 #Hill coefficient
Km = 40 #monomeres per cell
beta = 0.2
alpha0 = 0.2164 
alpha = 216.404

CmtetR0 = 1
Cmlambda0 = 4
CmlacI0 = 1
CptetR0 = 2
Cplambda0 = 1
CplacI0 = 1

In [13]:
#Calculating data for plots
    
def repressilator_model(t, data, alpha0, alpha, beta):
    m = data[:3]
    p = data[3:]
    dm0 = - m[0] + alpha/(1+p[2]**n) + alpha0
    dm1 = - m[1] + alpha/(1+p[0]**n) + alpha0
    dm2 = - m[2] + alpha/(1+p[1]**n) + alpha0
    dp0 = - beta*(p[0] - m[0])
    dp1 = - beta*(p[1] - m[1])
    dp2 = - beta*(p[2] - m[2])
    return [dm0,dm1,dm2,dp0,dp1,dp2]
    
t0 = 0.0
tf = 100.0
time = np.linspace(0.0,100.0,1000)
initval = np.array([CmtetR0,Cmlambda0,CmlacI0,CptetR0,Cplambda0,CplacI0])
res = solve_ivp(repressilator_model, (t0,tf), initval, method='RK45', t_eval=time, args=(alpha0,alpha,beta))
res = dict(res)
time = res['t']
y = res['y']

ylacI = y[5]
ytetR = y[3]
ylambda = y[4]

fig = plt.figure(title="Repressilator model", legend_location="top-left")

line_chart = plt.plot(x=time, y=[ylacI, ytetR, ylambda],
                     labels=["LacI","TetR", "cI"], colors=['blue', 'green', 'red'],
                     display_legend=True)

plt.xlabel("Time/mRNA half-life time")
plt.ylabel("Protein concentration, in units of Km")

In [14]:
#Callback
def update_plot(b, a, a0, CmlacI0, CmtetR0, CmcI0, CplacI0, CptetR0, CpcI0):
    t0 = 0.0
    tf = 100.0
    time = np.linspace(0.0,100.0,1000)
    initval = np.array([CmtetR0,CmcI0,CmlacI0,CptetR0,CpcI0,CplacI0])
    res = solve_ivp(repressilator_model, (t0,tf), initval, method='RK45', t_eval=time, args=(a0,a,b))
    res = dict(res)
    time = res['t']
    y = res['y']
    ylacI = y[5]
    ytetR = y[3]
    ylambda = y[4]
    line_chart.y = [ylacI, ytetR, ylambda]

In [15]:
#widgets
wdj = widgets.interactive(update_plot, b = widgets.FloatText(value=beta, description="beta:", disabled=False),
a = widgets.FloatText(value=alpha, description="alpha:", disabled=False),
a0 = widgets.FloatText(value=alpha0, description="alpha0:", disabled=False),
CmlacI0 = widgets.FloatText(value=CmlacI0, description="CmlacI0:", disabled=False),
CmtetR0 = widgets.FloatText(value=CmtetR0, description="CmtetR0:", disabled=False),
CmcI0 = widgets.FloatText(value=Cmlambda0, description="Cmlambda0:", disabled=False),
CplacI0 = widgets.FloatText(value=CplacI0, description="CplacI0:", disabled=False),
CptetR0 = widgets.FloatText(value=CptetR0, description="CptetR0:", disabled=False),
CpcI0 = widgets.FloatText(value=Cplambda0, description="Cplambda0:", disabled=False))

In [16]:
widgets.HBox([fig, wdj])